# Age prediction from MRI features

In [2]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import visualize, create_dataset_age, create_dataset_mri, cv
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, r2_score, mean_absolute_error
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from tensorflow.python.keras.activations import tanh
from sklearn.preprocessing import OneHotEncoder

#TF
import tensorflow as tf
from tensorflow.python import keras

/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/camillca/anaconda3/envs/dslab/lib/pyt

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Create dataset

In [4]:
target = 'Age'
# use all MRI high-level features, no DTI
data = create_dataset_mri(SCORE = target)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01', 'DX_01_Sub'], inplace=True)
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

In [9]:
column_names = X_train.columns

## Fix a baseline (mean age)

In [10]:
# Baseline
mean = np.mean(y_train)
print('mean_age = {}'.format(mean))
baseline_MSE = sum((mean - y_train)**2)/len(y_train)
print('baseline_MSE = {}'.format(baseline_MSE))

mean_age = 10.696383804093566
baseline_MSE = 13.820677264194652


# Explore different regressors and feature selection procedures

## SVR with no feature selection

In [11]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', SVR())])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [0.5, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5, 0.7],
            'regression__kernel' : ['rbf']}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Outer CV
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    5.8s finished


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    4.6s finished


Fitting 5 folds for each of 12 candidates, totalling 60 fits


...

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    4.7s finished



Average expected test MSE: 4.65245163796368

Average expected test r2: 0.6599854189578117

Average expected test mae: 1.6457929337893766
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    8.2s finished


Selected hyperparameters: {'regression__C': 2, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
True test error: 5.286587630228757
Test r2: 0.5886346387080388
Test mae: 1.7323091522236875


## SelectKBest + SVR


In [12]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [0.2, 0.5, 1, 0.5],
            'regression__epsilon' : [0.01, 0.1, 0.5, 0.7],
            'regression__kernel' : ['rbf'],
            'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 20, 40, 80, 95]}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring ='neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae=True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen   
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    3.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    4.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 80 candidates, totalling 400 fits


...

[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    3.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 5.215160306268113

Average expected test r2: 0.6167654472272817

Average expected test mae: 1.7747667878049218
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    4.8s


Selected hyperparameters: {'feat_select__k': 95, 'feat_select__score_func': <function f_regression at 0x7fe47bc527b8>, 'regression__C': 1, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
True test error: 5.7772353489898345
Test r2: 0.5504558568145675
Test mae: 1.8515552681479686


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:    6.3s finished


## ExtraTrees + SVR

In [13]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__C' : [0.05, 0.3, 1, 1.5],
            'regression__epsilon' : [0.05, 0.1, 0.5, 1],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   40.3s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   44.4s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   47.5s finished



Average expected test MSE: 4.976527255005277

Average expected test r2: 0.6361674289162469

Average expected test mae: 1.7197403109494815
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.1min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'regression__C': 1.5, 'regression__epsilon': 0.05, 'regression__kernel': 'rbf'}
True test error: 5.17687827314005
Test r2: 0.5971714553604003
Test mae: 1.7263418722384232


## extraTrees + random forest

In [14]:
pipe = Pipeline([ ('scaling', StandardScaler()),
                   ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 4, 5, 6],
              'regression__max_depth' : [2, 3, 6, 12],
            'regression__min_samples_leaf' : [1, 3, 5, 8]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  2.2min finished


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  2.2min finished


Fitting 5 folds for each of 80 candidates, totalling 400 fits


...

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  2.2min finished



Average expected test MSE: 6.378609978902338

Average expected test r2: 0.5352276008196395

Average expected test mae: 1.9777468304621766
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  3.4min finished


Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 100, 'regression__max_depth': 12, 'regression__min_samples_leaf': 8}
True test error: 7.30148266654077
Test r2: 0.4318495662657601
Test mae: 2.090678440564082


## RandomForest

In [15]:
pipe = Pipeline([ ('scaling', StandardScaler()),
                 
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__n_estimators':[100],
    'regression__max_depth' : [2, 3, 6, 12],
            'regression__min_samples_leaf' : [1,3, 5, 8]       
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   54.4s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   56.1s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   54.1s finished



Average expected test MSE: 6.059318920392617

Average expected test r2: 0.5579933878538172

Average expected test mae: 1.9338434445072068
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.2min finished


Selected hyperparameters: {'regression__max_depth': 12, 'regression__min_samples_leaf': 3, 'regression__n_estimators': 100}
True test error: 6.977413276057045
Test r2: 0.45706638498219077
Test mae: 2.012369593506228


## ExtraTrees + XGBoost

In [17]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 4, 5],
              'regression__booster__alpha' : [0.05, 0.2, 0.5],
              'regression__booster__max_depth' : [3, 4, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2= True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.6min finished


[22:10:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.6min finished


[22:12:01] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 48 candidates, totalling 240 fits


...

[22:15:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.5min finished


[22:16:46] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.348808590712174

Average expected test r2: 0.6085458746644272

Average expected test mae: 1.7943236284334163
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  2.3min finished


[22:19:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 100, 'regression__booster__alpha': 0.05, 'regression__booster__max_depth': 7}
True test error: 5.651026850959594
Test r2: 0.5602765214893493
Test mae: 1.830005840893555


## Lasso

In [18]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', Lasso())
])
# Prepare sets of parameters for gridsearch
parameters = {'regression__alpha' : [1, 1.2, 1.4, 1.7, 1.9, 2, 2.3, 2.5, 3, 3.5, 4, 6],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


...

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 10.052415299345125

Average expected test r2: 0.2649577510913282

Average expected test mae: 2.57756885271369
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Selected hyperparameters: {'regression__alpha': 1}
True test error: 9.886540701576447
Test r2: 0.2306983876751343
Test mae: 2.4604008547888543


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.6s finished


## SelectKBEST + XGBoost

In [19]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 30, 60, 100, 160],
              'regression__booster__alpha' : [0.05, 0.1, 0.6, 1],
              'regression__booster__max_depth' : [3, 4, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 297 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   20.2s finished


[22:19:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 291 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   20.2s finished


...

[22:20:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 290 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   20.1s finished


[22:20:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.102370363166074

Average expected test r2: 0.6264141239735699

Average expected test mae: 1.7486576020686595
Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   25.2s finished


[22:21:19] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__k': 160, 'feat_select__score_func': <function f_regression at 0x7fe47bc527b8>, 'regression__booster__alpha': 0.05, 'regression__booster__max_depth': 3}
True test error: 5.11952060396378
Test r2: 0.6016346289525005
Test mae: 1.7722031796078999


# XGBoost

In [5]:
pipe = Pipeline([ ('scaling', StandardScaler()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'regression__booster__alpha' : [0.05, 0.1, 0.6, 1],
              'regression__booster__max_depth' : [3, 4, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   16.4s finished


[17:12:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   19.5s finished


[17:12:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

[17:13:32] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   19.4s finished


[17:13:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 4.833082025277381

Average expected test r2: 0.6462714042254847

Average expected test mae: 1.7033556144664028
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   23.7s finished


[17:14:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'regression__booster__alpha': 0.05, 'regression__booster__max_depth': 3}
True test error: 4.920569871480459
Test r2: 0.6171155867407214
Test mae: 1.7168536330991107


# MRI + DTI

In [6]:
target = 'Age'
# use all MRI high-level features, no DTI
data = create_dataset_mri(SCORE = target, DTI = True)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01', 'DX_01_Sub'], inplace=True)

# labels and features
y = data[target]
X = data.drop([target, 'ID'], axis=1)
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

## ExtraTrees + SVR

In [21]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__C' : [0.3, 0.5, 0.8, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.5min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.5min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.6min finished



Average expected test MSE: 5.286903293764165

Average expected test r2: 0.6255066645600462

Average expected test mae: 1.8036839474373898
Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  2.5min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'median', 'regression__C': 2, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
True test error: 5.458047657991357
Test r2: 0.5752928157399706
Test mae: 1.7716924005218748


## ExtraTrees + XGBoost

In [22]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 4, 6],
              'regression__booster__alpha' : [0.05, 0.2, 0.5]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   53.9s finished


[22:32:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   54.0s finished


[22:33:46] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

[22:35:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   53.8s finished


[22:36:32] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.669313612441239

Average expected test r2: 0.5945505047237193

Average expected test mae: 1.8752272318164707
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.4min finished


[22:37:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 4, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'median', 'regression__booster__alpha': 0.5}
True test error: 5.948865859724257
Test r2: 0.5371007680512069
Test mae: 1.8171182853029888


# XGBoost

In [7]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__booster__alpha' : [0.05, 0.2, 0.5]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)
# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen  
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.5s finished


[17:16:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.4s finished


[17:16:42] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

[17:17:00] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    8.9s finished


[17:17:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.261738318862812

Average expected test r2: 0.6231606159888331

Average expected test mae: 1.8150220284418879
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   12.1s finished


[17:17:23] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'imputation__strategy': 'most_frequent', 'regression__booster__alpha': 0.05}
True test error: 5.79298233055769
Test r2: 0.5492305365862828
Test mae: 1.7747790001871746


## ExtraTrees + RandomForest

In [23]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 5, 8],
            'regression__max_depth' : [2, 3, 6],
            'regression__min_samples_leaf' : [1, 3, 8]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE

model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  1.8min finished


Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  1.8min finished


Fitting 5 folds for each of 81 candidates, totalling 405 fits


...

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  1.8min finished



Average expected test MSE: 6.902595882240765

Average expected test r2: 0.5082027752675551

Average expected test mae: 2.0440479009487307
Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  2.7min finished


Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 8, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__max_depth': 6, 'regression__min_samples_leaf': 8}
True test error: 7.519267921686595
Test r2: 0.4149030373451287
Test mae: 2.0371589129997525


# random Forest


In [24]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
            'regression__max_depth' : [2, 3, 6],
            'regression__min_samples_leaf' : [1, 3, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE

model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    7.3s finished


Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    7.2s finished


Fitting 5 folds for each of 27 candidates, totalling 135 fits


...

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    7.2s finished



Average expected test MSE: 7.184983230606642

Average expected test r2: 0.4878969027029368

Average expected test mae: 2.1026515917544186
Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    9.1s finished


Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__max_depth': 6, 'regression__min_samples_leaf': 6}
True test error: 6.61881137685613
Test r2: 0.48497028257035657
Test mae: 1.9707515785238645


## Lasso

In [7]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', Lasso())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__alpha' : [1, 1.2, 1.4, 1.7, 1.9, 2, 2.3, 2.5, 3, 3.5, 4, 6],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 9.877312142754754

Average expected test r2: 0.2954501815002074

Average expected test mae: 2.5463719001333525
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s


Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__alpha': 1}
True test error: 9.58189753364129
Test r2: 0.25440359330271034
Test mae: 2.4572954206670223


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.3s finished
